我们实现一下多输入通道互相关运算。 简而言之，我们所做的就是对每个通道执行互相关操作，然后将结果相加。

In [41]:
import torch
from d2l import torch as d2l

In [42]:
def corr2d_multi_in(X, K):
    # 先遍历X 和 K 的第0维度（通道维度），再把它们相加
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

我们可以构造与图中的值相对应的输入张量X和核张量K，以验证互相关运算的输出。

In [43]:
X = torch.tensor([ [[0.0, 1.0, 2.0],
                   [3.0, 4.0, 5.0],
                   [6.0, 7.0, 8.0]],
                   [[1.0, 2.0, 3.0],
                    [4.0, 5.0, 6.0],
                    [7.0, 8.0, 9.0]]
                  ])
K = torch.tensor([[[0.0, 1.0],
                   [2.0, 3.0]],
                  [[1.0, 2.0],
                   [3.0, 4.0]]])
corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

多输出通道
不论有多少输入通道，我们还只有一个输出通道。
直观地说，我们可以将每个通道看作对不同特征的响应。而现实可能更为复杂一些，因为每个通道不是独立学习的，而是为了共同使用而优化的。因此，多输出通道并不仅是学习多个单通道的检测器。

如下所示，我们实现一个计算多个通道的输出的互相关函数。

In [44]:
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

通过将核张量K与K+1（K中每个元素加）和K+2连接起来，构造了一个具有个输出通道的卷积核。

In [45]:
K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

下面，我们对输入张量X与卷积核张量K执行互相关运算。现在的输出包含个通道，第一个通道的结果与先前输入张量X和多输入单输出通道的结果一致。

In [46]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

 1 * 1卷积层
 作用：改变通道数目且不改变输入和输出的w和h

In [47]:
K.shape[0]

3

In [48]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0] # c_o = 3
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # 全连接层中的矩阵乘法
    Y = torch.matmul(K, K)
    return Y.reshape((c_o, h, w))

上述函数相当于先前实现的互相关函数corr2d_multi_in_out。让我们用一些样本数据来验证这一点。

In [49]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K).t()
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6

RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x3 and 2x3)